In [ ]:

# Install dependencies 
# pip install opencv-python 
%pip install mediapipe


# Import packages

In [21]:
import mediapipe as mp
import cv2
import numpy as np
import uuid
import os
import copy
import itertools
from collections import deque, Counter
import csv
import tensorflow as tf



# Initials MP Hands

In [22]:
mp_hands = mp.solutions.hands # hands model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# Detecting Hands Function
A list of 21 hand landmarks on the left hand. Each landmark consists of x, y and z. x and y are normalized to [0.0, 1.0] by the image width and height respectively.

In [23]:
def mediapipe_detection(image, model):
    image = cv2.flip(image, 1)                     # Mirror display
    debug_image = copy.deepcopy(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results ,debug_image

# Drawing Landmarks

In [42]:
def draw_landmarks(image, hand):
    # Draw Hand connections
    mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(33, 225, 225), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(33, 225, 225), thickness=2, circle_radius=2),
                             )

In [25]:
def draw_bounding_rect(use_brect, image, brect):
    if use_brect:
        # Outer rectangle
        cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[3]),
                     (0, 0, 0), 1)

    return image

In [29]:
def draw_info(image, mode, number):
    if 1 <= mode <= 2:
        cv2.putText(image, "Collecting Data ", (10, 90),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 1,
                   cv2.LINE_AA)
        if 0 <= number <= 9:
            cv2.putText(image, "NUM:" + str(number), (10, 110),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (33, 225, 225), 1,
                       cv2.LINE_AA)
    return image


In [30]:
def draw_info_text(image, brect,finger_gesture_text):
    cv2.rectangle(image, (brect[0], brect[1]), (brect[2], brect[1] - 22),
                 (0, 0, 0), -1)

    if finger_gesture_text != "":
        cv2.putText(image, "Hand Gesture:" + finger_gesture_text, (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv2.LINE_AA)
        cv2.putText(image, "Hand Gesture:" + finger_gesture_text, (10, 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2,
                   cv2.LINE_AA)

    return image

In [31]:
def draw_point_history(image, point_history):
    for index, point in enumerate(point_history):
        if point[0] != 0 and point[1] != 0:
            cv2.circle(image, (point[0], point[1]), 1 + int(index / 2),
                      (152, 251, 152), 2)

    return image

# Read labels

In [13]:
actions = ['goLeft', 'goRight', 'modeDiaPo','modeNormal','write']

# Draw The Rectangle Of Hand

In [32]:
def calc_bounding_rect(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_array = np.empty((0, 2), int)

    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1) 
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point = [np.array((landmark_x, landmark_y))]

        landmark_array = np.append(landmark_array, landmark_point, axis=0)

    x, y, w, h = cv2.boundingRect(landmark_array)

    return [x, y, x + w, y + h] # retuen the coords of 4 points of the rect

# Extract Keypoint Values of Hand


In [33]:
def calc_landmark_list(image, landmarks):
    image_width, image_height = image.shape[1], image.shape[0]

    landmark_point = []

    # Keypoint
    for _, landmark in enumerate(landmarks.landmark):
        landmark_x = min(int(landmark.x * image_width), image_width - 1)
        landmark_y = min(int(landmark.y * image_height), image_height - 1)

        landmark_point.append([landmark_x, landmark_y])

    return landmark_point # keypoint => [[x1,y1],[x2,y2]]

# Preprocess Data (Normalisation)

In our case  Coords

In [34]:
def pre_process_Keypoint_history(image, Keypoints_history):
    image_width, image_height = image.shape[1], image.shape[0]

    temporarly_Keypoints_history = copy.deepcopy(Keypoints_history)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, point in enumerate(temporarly_Keypoints_history):
        if index == 0:
            base_x, base_y = point[0], point[1] # point => [x,y]

        
        # Normalization of Coords
        temporarly_Keypoints_history[index][0] = (temporarly_Keypoints_history[index][0] -
                                        base_x) / image_width # x / image width
        temporarly_Keypoints_history[index][1] = (temporarly_Keypoints_history[index][1] -
                                        base_y) / image_height # y / image height

    # Convert to a one-dimensional list (Flatten the temporarly_Keypoints_history list to 1d list )
    temporarly_Keypoints_history = list(
        itertools.chain.from_iterable(temporarly_Keypoints_history))

    return temporarly_Keypoints_history

In [35]:
def pre_process_landmark(landmark_list):
    temporarly_landmark_list = copy.deepcopy(landmark_list)

    # Convert to relative coordinates
    base_x, base_y = 0, 0
    for index, landmark_point in enumerate(temporarly_landmark_list):
        if index == 0:
            base_x, base_y = landmark_point[0], landmark_point[1]

        temporarly_landmark_list[index][0] = temporarly_landmark_list[index][0] - base_x
        temporarly_landmark_list[index][1] = temporarly_landmark_list[index][1] - base_y

    # Convert to a one-dimensional list
    temporarly_landmark_list = list(
        itertools.chain.from_iterable(temporarly_landmark_list))

    # Normalization
    max_value = max(list(map(abs, temporarly_landmark_list)))

    def normalize_(n):
        return n / max_value

    temporarly_landmark_list = list(map(normalize_, temporarly_landmark_list))

    return temporarly_landmark_list

# Save Collected Coords

In [36]:
def logging_csv(number, mode, point_history_list):
    if mode == 0:
        pass
    if mode == 2 and (0 <= number <= 9):
        csv_path = 'Gestures_Presentation_Controle_DataSet/KeyPoints.csv'
        with open(csv_path, 'a', newline="") as f:
            writer = csv.writer(f)
            writer.writerow([number, *point_history_list])
    return

# Choise of Mode

In [37]:
def select_mode(key, mode):
    number = -1
    if 48 <= key <= 57:  # 0 ~ 9
        number = key - 48
    if key == 110:  # n normal
        mode = 0
    if key == 104:  # h trainig frame
        mode = 2
    return number, mode

In [38]:
# Parameters Initialisation 
history_length = 16 # lenght of list that takes max indexes of predections 
Keypoints_history = deque(maxlen=history_length)
Argmax_list = deque(maxlen=history_length)
mode = 2 #mode Normal 
use_boundary_recttangle = True

In [45]:
# Camera preparation
cap = cv2.VideoCapture(0)


# Set mediapipe model 
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5,max_num_hands=1) as hands: 

    while cap.isOpened():

       # Process Key (ESC: end) 
       key = cv2.waitKey(10)
       if key == 27:  # ESC
          break
       number, mode = select_mode(key, mode)

       # Camera capture #####################################################
       ret, frame = cap.read()
       if not ret:
          break

       # Make detections
       image, results, debug_image = mediapipe_detection(frame, hands)

       if results.multi_hand_landmarks is not None:
            for num, hand_landmarks in enumerate(results.multi_hand_landmarks):
                
                # Bounding box calculation
                brect = calc_bounding_rect(debug_image, hand_landmarks)

                # Landmark calculation
                landmark_list = calc_landmark_list(debug_image, hand_landmarks)

                # Conversion to relative coordinates / normalized coordinates
                pre_processed_landmark_list = pre_process_landmark(
                    landmark_list)
                pre_processed_Keypoints_list = pre_process_Keypoint_history(
                    debug_image, Keypoints_history)
                
                # Write to the dataset file
                logging_csv(number, mode, pre_processed_Keypoints_list)

                # Drawing part
                debug_image = draw_bounding_rect(use_boundary_recttangle, debug_image, brect)
                debug_image = draw_landmarks(debug_image, hand_landmarks)

                debug_image = draw_info_text(
                    debug_image,
                    brect,
                    "boussaid",
                )

       else :
         Keypoints_history.append([0, 0])

       debug_image = draw_info(debug_image, mode, number)
       debug_image = draw_point_history(debug_image, Keypoints_history)



       # Screen reflection 
       cv2.imshow('Hand Gesture Recognition', debug_image)


cap.release()
cv2.destroyAllWindows()
        

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:971: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'


# **Build and Train The Model**

**import necessary package**

In [ ]:
from sklearn.model_selection import train_test_split

**input shape**

In [ ]:
TIME_STEPS = 16
DIMENSION = 2

**Load DataSet and  Features & lables**

In [ ]:
dataset = 'Gestures_Presentation_Controle_DataSet/KeyPoints.csv'
model_save_path = 'Gestures_classifier.hdf5'

In [ ]:
X_features  = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (TIME_STEPS * DIMENSION) + 1)))
Y_lables  = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

# split features ana lables into train and test
RANDOM_SEED = 42
X_train, X_test, y_train, y_test = train_test_split(X_features, Y_lables, train_size=0.75, random_state=RANDOM_SEED)

**Define a Callback to use for EarlyStopping**

In [ ]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)

# callback for earlyStopping
Earlystopping_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

**Build The Model**

In [ ]:
# Modl Param
use_lstm = False
model = None
NUM_CLASSES = 4

In [ ]:
if use_lstm:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION, )),
        tf.keras.layers.Reshape((TIME_STEPS, DIMENSION), input_shape=(TIME_STEPS * DIMENSION, )), 
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(16, input_shape=[TIME_STEPS, DIMENSION]),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])
else:
    model = tf.keras.models.Sequential([
        tf.keras.layers.InputLayer(input_shape=(TIME_STEPS * DIMENSION, )),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
    ])

In [ ]:
model.compile( optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'] )

**Train The Model**

In [ ]:
model.fit( X_train, y_train, epochs=1000, batch_size=128, validation_data=(X_test, y_test), callbacks=[cp_callback, Earlystopping_callback] )

**Model Summary**

In [ ]:
model.summary()

**Save Weights**


In [ ]:
model.save(model_save_path, include_optimizer=False)

**Test the model ( Make Predictions )**

In [ ]:
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))
print(actions[np.argmax(np.squeeze(predict_result))])


Evaluate The Model

Confusion matrix

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

# Convert to model for Tensorflow-Lite


**Save as a model dedicated to inference**


In [ ]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
tflite_save_path = 'Gestures_classifier.tflite'

**Transform model (quantization)**

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)  # converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

**Inference test**

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
# get input/output tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))